 # Align each tool by: 
   1. Moving each to a preset calibration location 
   2. Do something to mark the location
       - Print a crosshair first? 
       - Move the camera to the crosshair
   3. Set the camera as the zero point of the reference geometry, makes calibration easier 
Start simple with a manual alignment to a point, later on use the camera to refine

In [4]:
# Import motion 
import os 
import json 
# Add hardware folder to path 
import sys
#sys.path.append('../')
#from hardware import idsMotionClient 

### Initialize

In [7]:
# Connect to motion
#ids = idsMotionClient.MotionClient()

# Import settings
with open(os.path.join('/home/pi/hubgit/circuitprinter/src/config', 'machine_settings.json')) as f:
    
    data = json.load(f)

print(data)

{'tools': {'Pneumatic extruder 1': {'is_active': 1, 'last_offset:': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, 'Pneumatic extruder 2': {'is_active': 1, 'last_offset:': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, 'Stepper extruder 1': {'is_active': 1, 'last_offset:': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, 'tool_camera': {'is_active': 1, 'last_offset:': {'x': 0.0, 'y': 0.0, 'z': 13.0}}}, 'z_cal_location': {'x': 272.6, 'y': 173.1, 'z': 5.0}, 'rapid_height': 3, 'light_control_pin': 4, 'tip_height': -1.3}


### Start the camera

In [10]:
#from jetcam.usb_camera import USBCamera
import ipywidgets
from IPython.display import display
#from jetcam.utils import bgr8_to_jpeg
from uuid import uuid1
import os 
import picamera

In [ ]:
# Full reset of the camera
#!"echo 'jayhawk.1' | sudo -S systemctl restart nvargus-daemon && printf '\n'"
#!ls -ltrh /dev/video*
#image_path = os.path.join('./images/', str(uuid1()) + '.jpg') 

# Functions
def save_image(img, image_path):
    with open(image_path, 'wb') as f:
        f.write(image.value)

# Initialize camera
# Device 0 is the dinolite 
# camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0)

# Device 1 is the logitech
camera = USBCamera(width=640, height=480, capture_width=640, capture_height=480, capture_device=1)
image = camera.read()
print(camera.value.shape)
image_widget = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(image)
#display(image_widget)
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')


In [ ]:
camera.unobserve(update_image, names='value')

### Create a GUI

In [1]:
from ipywidgets import Button, HBox, VBox, Label, Output

In [2]:
# Define speeds 
fast = 20
slow = 2

output = Output()

speeds = [str(-fast), str(-slow), str(slow), str(fast)]
xitems = [Button(description=w) for w in speeds]
yitems = [Button(description=w) for w in speeds]
zitems = [Button(description=w) for w in speeds]

fiducial_1_btn = Button(description="Save fiducial 1")
fiducial_2_btn = Button(description="Save fiducial 2")

xbox = HBox([Label(value=" X Axis  "), xitems[0], xitems[1], xitems[2], xitems[3]])
ybox = HBox([Label(value=" Y Axis  "), yitems[0], yitems[1], yitems[2], yitems[3]])
zbox = HBox([Label(value=" Z Axis  "), zitems[0], zitems[1], zitems[2], zitems[3]])
fiducial_save_box = HBox( [Label(value="Fiducials"), fiducial_1_btn, fiducial_2_btn])
direction_control = VBox([xbox, ybox, zbox])


display(output, direction_control, fiducial_save_box)

Output()

### Motion control
Move to location

In [ ]:
# - - - - - - - - Motion control - - - - - - - - # 
def x_fast_neg(b):
    motion.moveRel((-fast,0,0))
def x_slow_neg(b):
    motion.moveRel((-slow,0,0))
def x_slow_pos(b):
    motion.moveRel((slow,0,0))
def x_fast_pos(b):
    motion.moveRel((fast,0,0))
    
def y_fast_neg(b):
    motion.moveRel((0,-fast,0))
def y_slow_neg(b):
    motion.moveRel((0,-slow,0))
def y_slow_pos(b):
    motion.moveRel((0,slow,0))
def y_fast_pos(b):    
    motion.moveRel((0,fast,0))
    
def z_fast_neg(b):
    motion.moveRel((0,0,-fast))
def z_slow_neg(b):
    motion.moveRel((0,0,-slow))
def z_slow_pos(b):
    motion.moveRel((0,0,slow))
def z_fast_pos(b):
    motion.moveRel((0,0,fast))

def btn1click(b):
    (fid1x, fid1y) = motion.currentLocation()
    
def btn2click(b):
    (fid2x, fid2y) = motion.currentLocation()

xitems[0].on_click(x_fast_neg)
xitems[1].on_click(x_slow_neg)
xitems[2].on_click(x_slow_pos)
xitems[3].on_click(x_fast_pos)

yitems[0].on_click(y_fast_neg)
yitems[1].on_click(y_slow_neg)
yitems[2].on_click(y_slow_pos)
yitems[3].on_click(y_fast_pos)

zitems[0].on_click(z_fast_neg)
zitems[1].on_click(z_slow_neg)
zitems[2].on_click(z_slow_pos)
zitems[3].on_click(z_fast_pos) 

fiducial_1_btn.onclick(btn1click)
fiducial_2_btn.onclick(btn2click)

In [ ]:
### Save the location 